# Example: Multi-Gate River System

This notebook demonstrates a simulation of a river system with multiple controlled gates. The system consists of a `Reservoir`, a `RiverChannel`, and two `Gates`.

This example uses the centralized control capabilities of the `SimulationHarness`. Instead of autonomous agents, `PIDController` instances are added directly to the harness, which manages their execution. This is suitable for simpler control schemes or for simulating legacy control systems.

## Control Scheme

There are two independent PID controllers:

1.  **Reservoir Level Control**: A controller adjusts `gate_1`'s opening to maintain the `water_level` in `reservoir_1` at a setpoint of **18.0m**.
2.  **Channel Volume Control**: A second controller adjusts `gate_2`'s opening to maintain the `volume` of water in `channel_1` at a setpoint of **400,000 m^3**.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from swp.simulation_identification.physical_objects.reservoir import Reservoir
from swp.simulation_identification.physical_objects.gate import Gate
from swp.simulation_identification.physical_objects.river_channel import RiverChannel
from swp.local_agents.control.pid_controller import PIDController
from swp.core_engine.testing.simulation_harness import SimulationHarness

# 1. Define the components
reservoir = Reservoir(name="reservoir_1", initial_state={'volume': 25e6, 'water_level': 15.0}, parameters={'surface_area': 1.5e6})
gate1 = Gate(name="gate_1", initial_state={'opening': 0.2}, parameters={'max_rate_of_change': 0.05, 'discharge_coefficient': 0.6, 'width': 10, 'max_opening': 1.0})
channel = RiverChannel(name="channel_1", initial_state={'volume': 5e5, 'water_level': 5.0, 'outflow': 50}, parameters={'k': 0.0001, 'length': 5000})
gate2 = Gate(name="gate_2", initial_state={'opening': 0.5}, parameters={'max_rate_of_change': 0.05, 'discharge_coefficient': 0.6, 'width': 10, 'max_opening': 1.0})

# 2. Define the controllers
controller1 = PIDController(Kp=0.2, Ki=0.01, Kd=0.05, setpoint=18.0, min_output=0.0, max_output=1.0)
controller2 = PIDController(Kp=-1e-5, Ki=-1e-7, Kd=-1e-6, setpoint=4e5, min_output=0.0, max_output=1.0)

# 3. Set up the Simulation Harness
simulation_config = {'duration': 1000, 'dt': 1.0}
harness = SimulationHarness(config=simulation_config)

# 4. Add components and controllers
harness.add_component(reservoir)
harness.add_component(gate1)
harness.add_component(channel)
harness.add_component(gate2)

harness.add_controller(controller_id="res_level_ctrl", controller=controller1, controlled_id="gate_1", observed_id="reservoir_1", observation_key="water_level")
harness.add_controller(controller_id="chan_vol_ctrl", controller=controller2, controlled_id="gate_2", observed_id="channel_1", observation_key="volume")

# 5. Define Topology
harness.add_connection("reservoir_1", "gate_1")
harness.add_connection("gate_1", "channel_1")
harness.add_connection("channel_1", "gate_2")

# 6. Build and run the simulation
import sys
original_stdout = sys.stdout
with open('simulation_log.txt', 'w') as f:
    sys.stdout = f
    harness.build()
    harness.run_simulation()
sys.stdout = original_stdout

print("Multi-gate river simulation complete.")

## Results and Visualization

The plots below show the state of the controlled variables (reservoir level and channel volume) and the control actions (gate openings).

In [ ]:
# Extract data from history
time = [h['time'] for h in harness.history]
res_levels = [h['reservoir_1']['water_level'] for h in harness.history]
chan_volumes = [h['channel_1']['volume'] for h in harness.history]
g1_openings = [h['gate_1']['opening'] for h in harness.history]
g2_openings = [h['gate_2']['opening'] for h in harness.history]

# Create a DataFrame
df = pd.DataFrame({
    'Time': time,
    'Reservoir Level': res_levels,
    'Channel Volume': chan_volumes,
    'Gate 1 Opening': g1_openings,
    'Gate 2 Opening': g2_openings
})

print(df.head())

# Plot the results
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(12, 15), sharex=True)

# Controlled Variables
ax1.plot(df['Time'], df['Reservoir Level'], label='Reservoir Level')
ax1.axhline(y=18.0, color='r', linestyle='--', label='Reservoir Setpoint (18.0m)')
ax1.set_ylabel('Water Level (m)')
ax1.set_title('Multi-Gate River Simulation Results')
ax1.grid(True)
ax1.legend()

ax2.plot(df['Time'], df['Channel Volume'], label='Channel Volume', color='orange')
ax2.axhline(y=4e5, color='purple', linestyle='--', label='Channel Setpoint (4e5 m^3)')
ax2.set_ylabel('Volume (m^3)')
ax2.grid(True)
ax2.legend()

# Control Actions (Gate Openings)
ax3.plot(df['Time'], df['Gate 1 Opening'], label='Gate 1 Opening')
ax3.plot(df['Time'], df['Gate 2 Opening'], label='Gate 2 Opening')
ax3.set_xlabel('Time (s)')
ax3.set_ylabel('Gate Opening (%)')
ax3.grid(True)
ax3.legend()

plt.tight_layout()
plt.show()